In [4]:
from openai import AsyncOpenAI
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
import asyncio
from typing import Dict
from IPython.display import display, Markdown
from dotenv import load_dotenv

In [8]:
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [9]:
if OPENAI_API_KEY:
    print(f"OPENAI_API_KEY Key exists and begins {OPENAI_API_KEY[:2]}")
else:
    print("OPENAI_API_KEY Key not set (and this is optional)")

OPENAI_API_KEY Key exists and begins sk


In [10]:
model="gpt-4o-mini"

### OpenAI Hosted Tools
OpenAI Agents SDK includes the following hosted tools:

- The **WebSearchTool** lets an agent search the web.
-The **FileSearchTool** allows retrieving information from your OpenAI Vector Stores.
-The **ComputerTool** allows automating computer use tasks like taking screenshots and clicking.

Important note - API charge of WebSearchTool
This is costing me 2.5 cents per call for OpenAI WebSearchTool.  OpenAI can sometimes charge for multiple searches for a single call, so it could sometimes cost more than 2.5 cents per call.  

Costs are here: https://platform.openai.com/docs/pricing#web-search

* **Using WebSearchTool**

In [11]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. Limit the search results to 2. The summary must be less than 100 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model=model,
    model_settings=ModelSettings(tool_choice="required"),
)

In [12]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

In 2025, several AI agent frameworks have emerged, each offering unique capabilities:

- **Amazon Bedrock AgentCore**: AWS's platform simplifies AI agent development with modular services like AgentCore Runtime for scalable deployment, AgentCore Memory for context management, and AgentCore Identity for secure access. ([techradar.com](https://www.techradar.com/pro/aws-looks-to-super-charge-ai-agents-with-amazon-bedrock-agentcore?utm_source=openai))

- **LangChain**: A Python-based framework enabling developers to build custom LLM applications with features like prompt chaining, model integration, and external tool support. ([medium.com](https://medium.com/%40elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239?utm_source=openai))

- **LangGraph**: An extension of LangChain, LangGraph focuses on stateful, graph-based agent workflows, supporting complex interactions and branching logic. ([linkedin.com](https://www.linkedin.com/pulse/top-5-frameworks-building-ai-agents-2025-sahil-malhotra-wmisc?utm_source=openai))

- **AutoGen**: Developed by Microsoft, AutoGen facilitates multi-agent systems and code automation through conversation-based coordination and built-in agents. ([medium.com](https://medium.com/%40elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239?utm_source=openai))

- **CrewAI**: This framework emphasizes role-based AI agents, ideal for collaborative problem-solving environments requiring diverse expertise. ([linkedin.com](https://www.linkedin.com/pulse/top-5-frameworks-building-ai-agents-2025-sahil-malhotra-wmisc?utm_source=openai))

- **Eliza**: A Web3-friendly AI agent operating system that integrates seamlessly with blockchain applications, enabling efficient deployment of decentralized AI agents. ([arxiv.org](https://arxiv.org/abs/2501.06781?utm_source=openai))

- **AutoAgent**: A fully automated, zero-code framework for LLM agents, allowing users to create and deploy agents through natural language instructions. ([arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai))

These frameworks cater to diverse needs, from scalable LLM applications to collaborative multi-agent systems and Web3 integration. 

**We will now use Structured Outputs, and include a description of the fields**

In [13]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 2

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
# With massive thanks to student Wes C. for discovering and fixing a nasty bug with this!

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")
    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model=model,
    output_type=WebSearchPlan,
)

In [14]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find the most current information and trends about AI agent frameworks specifically in 2025, including new technologies, advancements, and popular platforms.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To explore detailed comparisons of different AI frameworks, including their capabilities, applications, and user experiences as of 2025.', query='AI agent frameworks comparison 2025')]


* Adding email tool and agent

In [15]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    print(f"Sending email with subject '{subject}' and body:\n{html_body}")
    return {"status": "success"}

send_email

FunctionTool(name='send_email', description='Send out an email with the given subject and HTML body', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x12a7ce020>, strict_json_schema=True, is_enabled=True)

In [16]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model=model,
)

In [17]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 2-3 pages of content, at least 300 words."
)

class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")
    markdown_report: str = Field(description="The final report")
    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model=model,
    output_type=ReportData,
)

* The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [18]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

* The next 2 functions write a report and email it

In [19]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

* Show time!!! - Putting all these together

In [20]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    print(search_plan)
    search_results = await perform_searches(search_plan)
    print(search_results)
    report = await write_report(query, search_results)
    await send_email(report)
    print("Hooray!")

Starting research...
Planning searches...
Will perform 2 searches
searches=[WebSearchItem(reason='To find comprehensive information on emerging AI agent frameworks that have been developed or released in 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To gather insights from industry experts and news articles regarding any trends or notable developments in AI agent technologies in 2025.', query='emerging AI technologies 2025')]
Searching...
Finished searching
['In 2025, several AI agent frameworks have emerged, enhancing the development of intelligent systems:\n\n- **Amazon Bedrock AgentCore**: Introduced by AWS, it offers a suite of tools for secure and scalable AI agent deployment, including secure API access and controlled web browsing. ([itpro.com](https://www.itpro.com/cloud/cloud-computing/three-of-the-biggest-announcements-from-aws-summit-new-york?utm_source=openai))\n\n- **OpenAI ChatGPT Agent**: Launched in July 2025, this tool autonomously performs com